In [35]:
import gpt4all
from gpt4all import GPT4All
from deep_translator import GoogleTranslator
import asyncio
import sys
from gtts import gTTS

In [2]:
dir(gpt4all)

['Embed4All',
 'GPT4All',
 'LLModel',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'gpt4all',
 'pyllmodel']

In [3]:
import os

In [4]:
from gpt4all import GPT4All

In [5]:
modelPath = os.getcwd()

In [6]:
print(modelPath)

C:\Users\Alaa AI\Python Projects\Chatbots\GPT by me - Ai Voice Translator & Chat


In [7]:
model = GPT4All(device = 'gpu' , model_name = "alaa_ai_model_gptj_v1.6.gguf" , model_path ='C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me - Ai Voice Translator & Chat/' , allow_download = False)

In [8]:
output = ''

In [9]:
target_lang = 'en'

In [10]:
# with model.chat_session():
#     input_text = "هل بإمكانك أن تخبرنى عن مرض adhd"
#     input_text = GoogleTranslator(source='auto', target='en').translate(input_text)
#     output = model.generate(prompt=input_text, max_tokens =  1000)
#     output = GoogleTranslator(source='auto', target = target_lang).translate(output)
#     print(output)

# Calling Voice Recognition -- Speech To Text 

Import Packages

In [10]:
import pyaudio
import wave

In [11]:
import requests
import json

Record Audio Function 

In [12]:
def record_audio(RECORD_SECONDS, WAVE_OUTPUT_FILENAME):
    #--------- SETTING PARAMS FOR OUR AUDIO FILE ------------#
    FORMAT = pyaudio.paInt16    # format of wave
    CHANNELS = 1                # no. of audio channels
    RATE = 44100                # frame rate
    CHUNK = 1024                # frames per audio sample
    #--------------------------------------------------------#
     
    # creating PyAudio object
    audio = pyaudio.PyAudio()
     
    # open a new stream for microphone
    # It creates a PortAudio Stream Wrapper class object
    stream = audio.open(format=FORMAT,channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)


    #----------------- start of recording -------------------#
    print("Listening...")

    # list to save all audio frames
    frames = []

    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        # read audio stream from microphone
        data = stream.read(CHUNK)
        # append audio data to frames list
        frames.append(data)

    #------------------ end of recording --------------------#   
    print("Finished recording.")
      
    stream.stop_stream()    # stop the stream object
    stream.close()          # close the stream object
    audio.terminate()       # terminate PortAudio

    #------------------ saving audio ------------------------#

    # create wave file object
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')

    # settings for wave file object
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))

    # closing the wave file object
    waveFile.close()

Read Audio Function

In [13]:
def read_audio(WAVE_FILENAME):
    # function to read audio(wav) file
    with open(WAVE_FILENAME, 'rb') as f:
        audio = f.read()
    return audio

Wit Access Token

In [14]:
# Wit speech API endpoint
API_ENDPOINT = 'https://api.wit.ai/speech'

# Wit.ai api access token
wit_access_token = 'S7JQYY2RA7ABGIFCOWGTUE6RDJSWHNF7'

Recognize Speech

In [15]:
text = ''

In [16]:
def RecognizeSpeech(AUDIO_FILENAME, num_seconds = 5):
    
    # record audio of specified length in specified audio file
    record_audio(num_seconds, AUDIO_FILENAME)
    
    # reading audio
    audio = read_audio(AUDIO_FILENAME)
    
    # defining headers for HTTP request
    headers = {'authorization': 'Bearer ' + wit_access_token,
               'Content-Type': 'audio/wav'}

    # making an HTTP post request
    resp = requests.post(API_ENDPOINT, headers = headers,
                         data = audio)
    
    # converting response content to JSON format
    
    data = json.loads(json.dumps(resp.text))
    data_text = str(data).splitlines()[-3].split(':')
    print(data_text[1])
    text = data_text[1]
#     print(data)
    # get text from data
#     text = data['text']
#     print("\nYou said: {}".format(text))
    # return the text
    return text

# Calling functions of Tokens

In [17]:
async def response4(g4a):
    # a better way to use an executor:
    with g4a.chat_session():
        # is_message_ended = True
        # while is_message_ended == True:
        print("\n")
        prompt =  RecognizeSpeech('myspeech.wav', 10) + "translate to" + target_lang
        # prompt = prompt.replace("\"", "") 
        # prompt = prompt.replace(",", "") 
        # if('See you' in prompt):
        #     break
        print("\nAlaa's robot:")
        generator = g4a.generate(prompt, max_tokens = 1000 , streaming = True)
        event_loop = asyncio.get_running_loop()
        has_tokens = True

        def consume(generator):
            nonlocal has_tokens
            try:
                return next(generator)
            except StopIteration:
                has_tokens = False

        while has_tokens:
            token = await event_loop.run_in_executor(None, consume, generator)
            if token is not None:
                yield token

In [18]:
def get_response_in_arabic(g4a):
    with g4a.chat_session():
        is_message_ended = True
        while is_message_ended == True:
            print("\n")
            prompt =  RecognizeSpeech('myspeech.wav', 10) + "translate to" + target_lang
            if(prompt == 'وداعا' or prompt == 'مع السلامة'):
                break
            input_text = prompt
            input_text = GoogleTranslator(source='auto', target='en').translate(input_text)
            output = g4a.generate(prompt=input_text, max_tokens =  1000)
            output = GoogleTranslator(source='auto', target = target_lang).translate(output)
            print("\nAlaa's robot:" , output)

In [27]:
output = []

In [29]:
if(target_lang == 'ar'):
    print("إذا أردت إنهاء الشات يرجى قول وداعاً او مع السلامة")
    get_response_in_arabic(model)
else:
    print("إذا أردت إنهاء الشات يرجى قول إلى اللقاء")
    async for token in response4(model):
        output.append(token)
        print(token, end='', flush=True)
    print()

إذا أردت إنهاء الشات يرجى قول إلى اللقاء


Listening...
Finished recording.
 "حبيبي كل سنة وانت طيب",

Alaa's robot:
 "My dear, you are good every year"




In [30]:
output

[' "',
 'My',
 ' dear',
 ',',
 ' you',
 ' are',
 ' good',
 ' every',
 ' year',
 '"',
 '\n',
 '\n']

In [32]:
print(" ".join(output))

 " My  dear ,  you  are  good  every  year " 
 



In [36]:
myobj = gTTS(text=" ".join(output), lang=target_lang, slow=False) 

In [37]:
myobj.save("welcome.mp3")

In [38]:
# Playing the converted file 
os.system("mpg321 welcome.mp3") 

1

In [ ]:
async def response_with_system_and_prompt_template(g4a , system_template , prompt_template):
    # a better way to use an executor:
    with g4a.chat_session(system_template , prompt_template):
        is_message_ended = True
        while is_message_ended == True:
            print("\n")
            prompt =  RecognizeSpeech('myspeech.wav', 10)
            if(prompt == 'goodbye' or prompt == 'exit'):
                break
            print("\nAlaa's robot:")
            generator = g4a.generate(prompt, max_tokens = 1000 , streaming = True)
            event_loop = asyncio.get_running_loop()
            has_tokens = True
    
            def consume(generator):
                nonlocal has_tokens
                try:
                    return next(generator)
                except StopIteration:
                    has_tokens = False
    
            while has_tokens:
                token = await event_loop.run_in_executor(None, consume, generator)
                if token is not None:
                    yield token

In [ ]:
system_template = 'I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations.'
prompt_template = 'Human: %1\nASSISTANT: \n'

In [ ]:
print("If you want to exit or end the chat ... Say 'goodbye' or 'exit'")
async for token in response_with_system_and_prompt_template(model , system_template , prompt_template):
    print(token, end='', flush=True)
print()